# Train a proprioception-tuned CNN

We create a sensor processing model using CNN-based visual encoding finetuned with proprioception.

We create an encoding for the robot starting from a pretrained CNN model. As the feature vector of this is still large (eg 512 * 7 * 7), we reduce this to the encoding with an MLP. 

We finetune the encoding with information from proprioception.  

The sensor processing object associated with the network trained like this is in sensorprocessing/sp_propriotuned_cnn.py

In [1]:
import sys

sys.path.append("..")


from settings import Config

import pathlib
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from behavior_cloning.demo_to_trainingdata import BCDemonstration
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, ResNetProprioTunedRegression
from robot.al5d_position_controller import RobotPosition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# The experiment/run we are going to run: the specified model will be created
# try:
#     import papermill as pm
#     run
# except NameError:
#     run = "default_value"  # Fallback if not provided
# print(f"Running with configuration: {run}")

experiment = "sensorprocessing_propriotuned_cnn"
# run = "vgg19_128"
# run = "resnet50_128"
# run = "vgg19_256"
run = "resnet50_256"

exp = Config().get_experiment(experiment, run)

Loading pointer config file: /home/ssheikholeslami/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/ssheikholeslami/SaharaBerryPickerData/settings-sahara.yaml


No system dependent experiment file
 /home/ssheikholeslami/SaharaBerryPickerData/experiments-Config/sensorprocessing_propriotuned_cnn/resnet50_256_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/resnet50_256 successfully loaded


### Create regression training data (image to proprioception)
The training data (X, Y) is all the pictures from a demonstration with the corresponding proprioception data. 

In [3]:
def load_images_as_proprioception_training(task, proprioception_input_file, proprioception_target_file):
    """Loads all the images of a task, and processes it as two tensors as input and target data for proprioception training.
    Caches the processed results into the input and target file pointed in the config. Remove those files to recalculate
    """
    retval = {}
    if proprioception_input_file.exists():
        retval["inputs"] = torch.load(proprioception_input_file, weights_only=True)
        retval["targets"] = torch.load(proprioception_target_file, weights_only=True)
    else:
        demos_dir = pathlib.Path(Config()["demos"]["directory"])
        task_dir = pathlib.Path(demos_dir, "demos", task)

        inputlist = []
        targetlist = []

        for demo_dir in task_dir.iterdir():
            if not demo_dir.is_dir():
                pass
            bcd = BCDemonstration(demo_dir, sensorprocessor=None)
            for i in range(bcd.trim_from, bcd.trim_to):
                sensor_readings, _ = bcd.get_image(i)
                inputlist.append(sensor_readings[0])
                a = bcd.get_a(i)
                rp = RobotPosition.from_vector(a)
                anorm = rp.to_normalized_vector()
                targetlist.append(torch.from_numpy(anorm))

        retval["inputs"] = torch.stack(inputlist)
        retval["targets"] = torch.stack(targetlist)
        torch.save(retval["inputs"], proprioception_input_file)
        torch.save(retval["targets"], proprioception_target_file)

    # Separate the training and validation data.
    # We will be shuffling the demonstrations
    length = retval["inputs"].size(0)
    rows = torch.randperm(length)
    shuffled_inputs = retval["inputs"][rows]
    shuffled_targets = retval["targets"][rows]

    training_size = int( length * 0.67 )
    retval["inputs_training"] = shuffled_inputs[1:training_size]
    retval["targets_training"] = shuffled_targets[1:training_size]

    retval["inputs_validation"] = shuffled_inputs[training_size:]
    retval["targets_validation"] = shuffled_targets[training_size:]

    return retval

In [4]:
task = exp["proprioception_training_task"]
proprioception_input_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_input_file"])
proprioception_target_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_target_file"])

tr = load_images_as_proprioception_training(
    task, proprioception_input_file, proprioception_target_file)
inputs_training = tr["inputs_training"]
targets_training = tr["targets_training"]
inputs_validation = tr["inputs_validation"]
targets_validation = tr["targets_validation"]

Cameras found: ['dev2']
There are 496 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
Cameras found: ['dev2']
There are 541 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
Cameras found: ['dev2']
There are 656 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
Cameras found: ['dev2']
There are 460 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']


### Create a model that performs proprioception regression

In [5]:

if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp, device)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
else:
    raise Exception(f"Unknown model {exp['model']}")

if exp['loss'] == 'MSELoss':
    criterion = nn.MSELoss()
elif exp['loss'] == 'L1Loss':
    criterion = nn.L1Loss()

optimizer = optim.Adam(model.parameters(), lr=exp['learning_rate'])

/apps/anaconda/anaconda-2023.09/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/apps/anaconda/anaconda-2023.09/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Create DataLoaders for batching
batch_size = exp['batch_size']
train_dataset = TensorDataset(inputs_training, targets_training)
test_dataset = TensorDataset(inputs_validation, targets_validation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
def train_and_save_proprioception_model(model, criterion, optimizer, modelfile, device="cpu", epochs=20):
    """Trains and saves the proprioception model
    FIXME: must have parameters etc to investigate alternative models.
    """

    model = model.to(device)
    criterion = criterion.to(device)
    # Training loop
    num_epochs = epochs
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            # Forward pass
            # print("batchX immediately after for loobatch_X.device" )
            # predictions = model(batch_X)
            predictions = model.forward(batch_X)
            #print(batch_y.device)
            # I don't understand this
            # print(batch_X.device)
            loss = criterion(predictions, batch_y)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if (epoch + 1) % 1 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

    # Evaluate the model
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            test_loss += loss.item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')
    torch.save(model.state_dict(), modelfile)

In [8]:
modelfile = pathlib.Path(
    exp["data_dir"], exp["proprioception_mlp_model_file"])
epochs = exp["epochs"]
if modelfile.exists():
    model.load_state_dict(torch.load(modelfile))
else:
    train_and_save_proprioception_model(model, criterion, optimizer, modelfile, device=device, epochs=epochs)

Epoch [1/300], Loss: 0.0969
Epoch [2/300], Loss: 0.0547
Epoch [3/300], Loss: 0.0515
Epoch [4/300], Loss: 0.0438
Epoch [5/300], Loss: 0.0422
Epoch [6/300], Loss: 0.0397
Epoch [7/300], Loss: 0.0371
Epoch [8/300], Loss: 0.0331
Epoch [9/300], Loss: 0.0312
Epoch [10/300], Loss: 0.0288
Epoch [11/300], Loss: 0.0309
Epoch [12/300], Loss: 0.0274
Epoch [13/300], Loss: 0.0282
Epoch [14/300], Loss: 0.0265
Epoch [15/300], Loss: 0.0242
Epoch [16/300], Loss: 0.0243
Epoch [17/300], Loss: 0.0253
Epoch [18/300], Loss: 0.0230
Epoch [19/300], Loss: 0.0229
Epoch [20/300], Loss: 0.0215
Epoch [21/300], Loss: 0.0212
Epoch [22/300], Loss: 0.0196
Epoch [23/300], Loss: 0.0213
Epoch [24/300], Loss: 0.0177
Epoch [25/300], Loss: 0.0186
Epoch [26/300], Loss: 0.0179
Epoch [27/300], Loss: 0.0167
Epoch [28/300], Loss: 0.0177
Epoch [29/300], Loss: 0.0157
Epoch [30/300], Loss: 0.0163
Epoch [31/300], Loss: 0.0159
Epoch [32/300], Loss: 0.0139
Epoch [33/300], Loss: 0.0161
Epoch [34/300], Loss: 0.0131
Epoch [35/300], Loss: 0

In [9]:
# print(model.resnet.fc.in_features)